# Penny stock Analysis

In [21]:
import investpy
import yfinance as yf
import pandas as pd
import pytz
import time
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import requests.exceptions

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

In [19]:
# Get NASDAQ-listed companies
nasdaq_url = "ftp://ftp.nasdaqtrader.com/symboldirectory/nasdaqlisted.txt"
nasdaq_df = pd.read_csv(nasdaq_url, delimiter='|')
nasdaq_df = nasdaq_df[nasdaq_df['Test Issue'] == 'N']  # Remove test issues
nasdaq_df = nasdaq_df[:-1]  # Remove the last row (file details)
nasdaq_df = nasdaq_df[['Symbol', 'Security Name']]

# Get NYSE and other listed companies
other_url = "ftp://ftp.nasdaqtrader.com/symboldirectory/otherlisted.txt"
other_df = pd.read_csv(other_url, delimiter='|')
other_df = other_df[other_df['Test Issue'] == 'N']  # Remove test issues
other_df = other_df[:-1]  # Remove the last row (file details)
other_df['Symbol'] = other_df['NASDAQ Symbol']
other_df = other_df[['Symbol', 'Security Name']]

all_listed_companies = pd.concat([nasdaq_df, other_df])

all_listed_companies = all_listed_companies.reset_index(drop=True)

# Save to CSV
all_listed_companies.to_csv('all_listed_companies.csv', index=False)

In [20]:
all_listed_companies

,Symbol,Security Name
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...
1,AACBR,Artius II Acquisition Inc. - Rights
2,AACBU,Artius II Acquisition Inc. - Units
3,AACG,ATA Creativity Global - American Depositary Sh...
4,AADR,AdvisorShares Dorsey Wright ADR ETF
...,...,...
11322,ZTO,ZTO Express (Cayman) Inc. American Depositary ...
11323,ZTR,Virtus Total Return Fund Inc.
11324,ZTS,Zoetis Inc. Class A Common Stock
11325,ZVIA,Zevia PBC Class A Common Stock


In [24]:
def penny_stock_identifier(df, price_threshold=5.0):
    """
    Identifies penny stocks from a dataframe of stock symbols based on price threshold.
    
    Args:
        df: Pandas DataFrame containing 'Symbol' and 'Security Name' columns
        price_threshold: Maximum price for a stock to be considered a penny stock (default: $5.00)
        
    Returns:
        DataFrame with 'Symbol', 'Security Name', and 'Price' columns for penny stocks
    """
    # Copy the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Convert Symbol column to string to avoid join errors
    result_df['Symbol'] = result_df['Symbol'].astype(str)
    
    # Define the retry decorator for the price fetching function
    @retry(
        stop=stop_after_attempt(5),
        wait=wait_exponential(multiplier=1, min=4, max=60),
        retry=retry_if_exception_type((
            requests.exceptions.RequestException,
            requests.exceptions.Timeout,
            requests.exceptions.ConnectionError,
            ValueError,
            TypeError
        ))
    )
    def get_price_with_retry(symbol):
        """Fetch price for a single symbol with retry logic for handling timeouts and exceptions"""
        try:
            data = yf.Ticker(symbol).history(period="1d")
            if not data.empty:
                return data['Close'].iloc[-1]
            return None
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            raise  # Re-raise the exception for tenacity to handle
    
    # Get current prices for all stocks in batches to avoid API limits
    prices = {}
    symbols = result_df['Symbol'].tolist()
    
    print(f"Fetching current prices for {len(symbols)} symbols...")
    
    # Process in batches of 100 symbols
    batch_size = 100
    
    for i in range(0, len(symbols), batch_size):
        batch = symbols[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1} of {(len(symbols) + batch_size - 1)//batch_size}...")
        
        # Convert batch to string for yfinance
        symbols_str = ' '.join(batch)
        
        try:
            # Fetch data for the batch
            data = yf.download(symbols_str, period="1d", progress=False)
            
            # Process the closing prices
            if isinstance(data.Close, pd.DataFrame):  # Multiple symbols
                for symbol in batch:
                    if symbol in data.Close.columns:
                        prices[symbol] = data.Close[symbol].iloc[-1]
                    else:
                        # Try individual request with retry for symbols that failed
                        try:
                            prices[symbol] = get_price_with_retry(symbol)
                        except Exception:
                            prices[symbol] = None
            else:  # Single symbol case
                if batch and not data.empty:
                    prices[batch[0]] = data.Close.iloc[-1]
                else:
                    # Try individual request with retry
                    try:
                        prices[batch[0]] = get_price_with_retry(batch[0])
                    except Exception:
                        prices[batch[0]] = None
            
            # Avoid hitting rate limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Batch error: {e}")
            # If batch fails, try individual symbols with retry
            for symbol in batch:
                try:
                    prices[symbol] = get_price_with_retry(symbol)
                except Exception:
                    prices[symbol] = None
    
    # Add price column to dataframe
    result_df['Price'] = result_df['Symbol'].map(prices)
    
    # Filter for penny stocks based on price threshold
    penny_stocks_df = result_df[result_df['Price'] < price_threshold].copy()
    
    # Remove stocks with no price data
    penny_stocks_df = penny_stocks_df.dropna(subset=['Price'])
    
    # Reset index
    penny_stocks_df = penny_stocks_df.reset_index(drop=True)
    
    print(f"Found {len(penny_stocks_df)} penny stocks under ${price_threshold:.2f}")
    
    return penny_stocks_df

In [ ]:
penny_stock_df = penny_stock_identifier(all_listed_companies, price_threshold=5.0)

In [27]:
penny_stock_df.to_csv('penny_stocks.csv')

In [28]:
penny_stock_df

,Symbol,Security Name,Price
0,AACBR,Artius II Acquisition Inc. - Rights,0.2399
1,AACG,ATA Creativity Global - American Depositary Sh...,0.9200
2,AAME,Atlantic American Corporation - Common Stock,1.7200
3,ABAT,American Battery Technology Company - Common S...,1.5500
4,ABCL,AbCellera Biologics Inc. - Common Shares,2.0300
...,...,...,...
2104,ZEPP,Zepp Health Corporation American depositary sh...,2.7800
2105,ZH,"Zhihu Inc. American Depositary Shares, each re...",4.0500
2106,ZKH,"ZKH Group Limited American Depositary Shares, ...",3.0000
2107,ZONE,CleanCore Solutions Inc. Class B Common Stock,1.7200
